In [1]:
import pandas as pd
from stage import fullPrint
from datetime import date
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [5]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

In [3]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
        df.to_pickle('stockData/nasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

1/5474
2/5474
3/5474
AACIW not found
4/5474
5/5474
AADR not found
6/5474
7/5474
8/5474
9/5474
10/5474
11/5474
12/5474
13/5474
AAXJ not found
14/5474
15/5474
16/5474
17/5474
18/5474
19/5474
20/5474
21/5474
22/5474
23/5474
24/5474
25/5474
26/5474
27/5474
28/5474
ACABU not found
29/5474
30/5474
31/5474
32/5474
ACAHW not found
33/5474
34/5474
35/5474
36/5474
ACBAW not found
37/5474
38/5474
39/5474
40/5474
41/5474
42/5474
43/5474
ACEVW not found
44/5474
45/5474
46/5474
47/5474
48/5474
49/5474
50/5474
51/5474
52/5474
ACKIT not found
53/5474
ACKIU not found
54/5474
ACKIW not found
55/5474
56/5474
57/5474
58/5474
59/5474
60/5474
61/5474
ACQRW not found
62/5474
63/5474
64/5474
65/5474
66/5474
67/5474
68/5474
ACTDW not found
69/5474
70/5474
71/5474
ACWI not found
72/5474
ACWX not found
73/5474
74/5474
75/5474
76/5474
77/5474
78/5474
79/5474
80/5474
ADER not found
81/5474
82/5474
ADERW not found
83/5474
84/5474
85/5474
86/5474
87/5474
ADILW not found
88/5474
89/5474
90/5474
91/5474
ADNWW not foun

In [3]:
import numpy as np
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,8]/df.iat[index-1,8]
def peakCheck(df, index):
    if index == 0:
        return df.iat[index,3]
    else:
        return max(df.iat[index-1,13], df.iat[index,3])
def troughCheck(df, index):
    if index == 0:
        return df.iat[index,3]
    elif df.iat[index,13]==df.iat[index,3]:
        return df.iat[index,3]
    else:
        return min(df.iat[index-1,14], df.iat[index,3])
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,3]/df.iat[index-30,3]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,3]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,3])

In [6]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
import os
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = stockDf['close'].rolling(window=104, closed='left').max().shift(1).fillna(0) 
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, 7] = calculatePeak(stockDf, numIndex, 260, 7)
            if numIndex < 104:
                stockDf.iat[numIndex, 13] = calculatePeak(stockDf, numIndex, 104, 13)
            stockDf.iat[stockDf.index.get_loc(index), stockDf.columns.get_loc('trough')] = troughCheck(stockDf,stockDf.index.get_loc(element.name))
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 25556.91it/s]


PCF


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 24981.38it/s]


HGLB


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 22837.62it/s]


...

ZTS


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 23164.61it/s]

ZUO



Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 24054.74it/s]


ZWS


In [37]:
fullPrint(pd.read_pickle('stockData/tickers/A.pkl')[['close','fiveYearHigh','peak']])

                 close  fiveYearHigh        peak
1999-11-15   28.880545     28.880545   28.880545
1999-11-22   29.461731     29.461731   29.461731
1999-11-29   31.831186     31.831186   31.831186
1999-12-06   32.010014     32.010014   32.010014
1999-12-13   32.859444     32.859444   32.859444
1999-12-20   35.586552     35.586552   35.586552
1999-12-27   55.302216     55.302216   55.302216
2000-01-03   46.494991     55.302216   55.302216
2000-01-10   48.909157     55.302216   55.302216
2000-01-17   49.177395     55.302216   55.302216
2000-01-24   48.685623     55.302216   55.302216
2000-01-31   54.542202     55.302216   55.302216
2000-02-07   53.916309     55.302216   55.302216
2000-02-14   67.060089     67.060089   67.060089
2000-02-21   77.342628     77.342628   77.342628
2000-02-28   77.253220     77.342628   77.342628
2000-03-06  101.573677    101.573677  101.573677
2000-03-13   86.641632    101.573677  101.573677
2000-03-20   85.836906    101.573677  101.573677
2000-03-27   74.3919